En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

In [ ]:
file_path = "farmers-protest-tweets-2021-2-4.json"

### Entendiendo generalmente los problemas

El challenge indica que debo presentar dos soluciones, una enfocada en memoria y otra en tiempo de ejecucion. Es decir:

Lo que necesito para la solución enfocada en el tiempo de ejecución es
reducir o minimizar el tiempo total que tarde la funcion


Lo que necesito para la solución enfocada en el tiempo de ejecución es
reducir o minimizar la cantidad de ram que utilice la funcion durante la ejecucion.


### Revisando la estrucutura de los datos

In [2]:
import json

def read_file(file_path):
    processed_data = []

    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            try:
                json_object = json.loads(line.strip())
                processed_data.append(json_object)
                break
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")

    return processed_data

output_path = "farmers-protest-tweets-2021-2-4.json"
json_data = read_file(file_path)
print(json_data[0].keys())


dict_keys(['url', 'date', 'content', 'renderedContent', 'id', 'user', 'outlinks', 'tcooutlinks', 'replyCount', 'retweetCount', 'likeCount', 'quoteCount', 'conversationId', 'lang', 'source', 'sourceUrl', 'sourceLabel', 'media', 'retweetedTweet', 'quotedTweet', 'mentionedUsers'])


voy identificar los campos que estare utilizando a lo largo de los 3 problemas



In [12]:
# q1
print('date', json_data[0]['date'])
print('user', json_data[0]['user'])
print('username', json_data[0]['user']['username'])

# q2 y q3
print('content', json_data[0]['content'])

date 2021-02-24T09:23:35+00:00
user {'username': 'ArjunSinghPanam', 'displayname': 'Arjun Singh Panam', 'id': 45091142, 'description': 'Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie', 'rawDescription': 'Global Citizen, Actor, Director: Sky is the roof above my head, the world is the road I travel, love is my food & mother earth is my bed. Roy in @CosmosMovie', 'descriptionUrls': [], 'verified': False, 'created': '2009-06-06T07:50:57+00:00', 'followersCount': 603, 'friendsCount': 311, 'statusesCount': 17534, 'favouritesCount': 4269, 'listedCount': 23, 'mediaCount': 1211, 'location': '', 'protected': False, 'linkUrl': 'https://www.cosmosmovieofficial.com', 'linkTcourl': 'https://t.co/3uaoV3gCt3', 'profileImageUrl': 'https://pbs.twimg.com/profile_images/1215541746492461056/3De61YoQ_normal.jpg', 'profileBannerUrl': 'https://pbs.twimg.com/profile_banners/45091142/1612601766', 'url

### En esta seccion se resolveran los problemas

#### Primer acercamiento general para resolver ambos problemas

**Acercamiento para resolver optimizando el tiempo de ejecucion**

Para reducir el tiempo de ejecucion podriamos cargar el dataset completo a memoria para que podamos acceder a la data con mayor rapidez.
Posteriormente podemos cargar esa informacion a un dataframe de pandas ya que las operaciones  se enceuntran optimizadas.

**Acercamiento para resolver optimizando el uso de ram**

Leer en streaming o linea por linea para evitar cargar en memoria todo el dataset, podemos lograr esto de alguna forma con iteradores o generadores de python asi solo tendresmos la informacion necesaria en memoria.


#### Pregunta 1


1. Las top 10 fechas donde hay más tweets. Mencionar el usuario (username) que más publicaciones tiene por cada uno de esos días. Debe incluir las siguientes funciones:
```python
def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
```
```python
Returns:
[(datetime.date(1999, 11, 15), "LATAM321"), (datetime.date(1999, 7, 15), "LATAM_CHI"), ...]
```

In [19]:
import pandas as pd
import datetime
from typing import List, Tuple

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    df = pd.read_json(file_path, lines=True, encoding='utf-8')

    df['date'] = pd.to_datetime(df['date']).dt.date

    df['username'] = df['user'].apply(lambda x: x['username'])

    top_dates = df['date'].value_counts().nlargest(10).index.tolist()

    result = []
    for date in top_dates:
        df_date = df[df['date'] == date]
        top_user = df_date['username'].value_counts().idxmax()
        result.append((date, top_user))

    return result

q1_time(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

Midiendo el tiempo de ejecucion:

In [24]:
import cProfile
import pstats
import io

pr = cProfile.Profile()

pr.enable()
result = q1_time(file_path)
pr.disable()

s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('cumulative' )
ps.print_stats(5)
print(s.getvalue())


         616539 function calls (615855 primitive calls) in 17.319 seconds

   Ordered by: cumulative time
   List reduced from 697 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   17.318    8.659 /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3512(run_code)
        2    0.000    0.000   17.318    8.659 {built-in method builtins.exec}
        1    0.721    0.721   17.318   17.318 <ipython-input-24-ca349d025676>:1(<cell line: 8>)
        1    0.076    0.076   16.597   16.597 <ipython-input-19-b6158fe0ac40>:5(q1_time)
        1    0.022    0.022   16.126   16.126 /usr/local/lib/python3.10/dist-packages/pandas/io/json/_json.py:505(read_json)





Para la medir la memoria con memory-profiler necesito ejecutar el archivo de python desde la linea de comandos el codigo sera el mismo de arriba, unicamente agregare el decorador @profile.

Para ello creare otro archivo para asi no afectar a los ya existentes en el proyecto

In [28]:
!pip install memory-profiler
!python /content/q1_time_mp.py

Filename: /content/q1_time_mp.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     6    116.6 MiB    116.6 MiB           1   @profile
     7                                         def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
     8                                             # Read the NDJSON file into a pandas DataFrame
     9   1997.6 MiB   1881.0 MiB           1       df = pd.read_json(file_path, lines=True, encoding='utf-8')
    10                                             
    11   2001.0 MiB      3.4 MiB           1       df['date'] = pd.to_datetime(df['date']).dt.date
    12                                             
    13   2001.0 MiB      0.0 MiB      234815       df['username'] = df['user'].apply(lambda x: x['username'])
    14                                             
    15   2001.6 MiB      0.5 MiB           1       top_dates = df['date'].value_counts().nlargest(10).index.tolist()
    16                                         

In [35]:
import json
from typing import List, Tuple
from datetime import datetime
from collections import defaultdict

def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
    date_counts = defaultdict(int)
    user_counts_per_date = defaultdict(lambda: defaultdict(int))

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            tweet = json.loads(line)
            date_str = tweet['date'][:10]
            date = datetime.strptime(date_str, '%Y-%m-%d').date()
            username = tweet['user']['username']

            date_counts[date] += 1
            user_counts_per_date[date][username] += 1

    top_dates = sorted(date_counts.items(), key=lambda x: x[1], reverse=True)[:10]

    result = []
    for date, _ in top_dates:
        user_counts = user_counts_per_date[date]
        top_user = max(user_counts.items(), key=lambda x: x[1])[0]
        result.append((date, top_user))

    return result

q1_memory(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria'),
 (datetime.date(2021, 2, 19), 'Preetm91')]

In [26]:
import cProfile
import pstats
import io

pr = cProfile.Profile()

pr.enable()
result = q1_memory(file_path)
pr.disable()

s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('cumulative' )
ps.print_stats(5)
print(s.getvalue())


         4253073 function calls in 14.764 seconds

   Ordered by: cumulative time
   List reduced from 65 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000   14.764    7.382 /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3512(run_code)
        2    0.000    0.000   14.764    7.382 {built-in method builtins.exec}
        1    0.001    0.001   14.764   14.764 <ipython-input-26-4354b2f277ac>:1(<cell line: 8>)
        1    2.806    2.806   14.763   14.763 <ipython-input-22-230961d9bc42>:4(q1_memory)
   117407    0.285    0.000    7.372    0.000 /usr/lib/python3.10/json/__init__.py:299(loads)





In [29]:
!pip install memory-profiler
!python /content/q1_memory_mp.py

Filename: /content/q1_memory_mp.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     5     37.8 MiB     37.8 MiB           1   @profile
     6                                         def q1_memory(file_path: str) -> List[Tuple[datetime.date, str]]:
     7     37.8 MiB      0.0 MiB           1       import json
     8     37.8 MiB      0.0 MiB           1       from collections import defaultdict
     9     37.8 MiB      0.0 MiB           1       import datetime
    10                                         
    11     37.8 MiB      0.0 MiB           1       date_counts = defaultdict(int)
    12     42.7 MiB      0.0 MiB          27       user_counts_per_date = defaultdict(lambda: defaultdict(int))
    13                                         
    14     43.5 MiB      0.0 MiB           2       with open(file_path, 'r', encoding='utf-8') as f:
    15     43.5 MiB      0.6 MiB      117408           for line in f:
    16     43.5 MiB      3.1 MiB      117407           

##### **Observaciones**

El uso de memoria si disminuye considerablemente pasando de 2k MiB a solo 43 MiB. Sin embargo no parece haber gran diferencia en el tiempo de ejecución.

Pienso que una razon de esto puede ser la lectura del archivo, realizare unas modificaciones para probar una forma mas rapida de leerlo.

Intentare cargar los datos con una libreria para parsear el json mas rapidament

In [30]:
import pandas as pd
import orjson
from typing import List, Tuple

def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
    data = []
    with open(file_path, 'rb') as f:
        for line in f:
            tweet = orjson.loads(line)
            data.append({
                'date': tweet['date'],
                'username': tweet['user']['username']
            })

    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date']).dt.date

    date_counts = df['date'].value_counts()
    top_dates = date_counts.nlargest(10).index

    df_top_dates = df[df['date'].isin(top_dates)]
    user_date_counts = df_top_dates.groupby(['date', 'username']).size().reset_index(name='counts')

    idx = user_date_counts.groupby('date')['counts'].idxmax()
    top_users = user_date_counts.loc[idx]

    result = list(zip(top_users['date'], top_users['username']))

    return result

q1_time(file_path)

[(datetime.date(2021, 2, 12), 'RanbirS00614606'),
 (datetime.date(2021, 2, 13), 'MaanDee08215437'),
 (datetime.date(2021, 2, 14), 'rebelpacifist'),
 (datetime.date(2021, 2, 15), 'jot__b'),
 (datetime.date(2021, 2, 16), 'jot__b'),
 (datetime.date(2021, 2, 17), 'RaaJVinderkaur'),
 (datetime.date(2021, 2, 18), 'neetuanjle_nitu'),
 (datetime.date(2021, 2, 19), 'Preetm91'),
 (datetime.date(2021, 2, 20), 'MangalJ23056160'),
 (datetime.date(2021, 2, 23), 'Surrypuria')]

In [31]:
import cProfile
import pstats
import io

pr = cProfile.Profile()

pr.enable()
result = q1_time(file_path)
pr.disable()

s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('cumulative' )
ps.print_stats(5)
print(s.getvalue())


         477212 function calls (477079 primitive calls) in 3.849 seconds

   Ordered by: cumulative time
   List reduced from 737 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    3.849    1.924 /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3512(run_code)
        2    0.000    0.000    3.849    1.924 {built-in method builtins.exec}
        1    0.029    0.029    3.849    3.849 <ipython-input-31-ca349d025676>:1(<cell line: 8>)
        1    0.992    0.992    3.820    3.820 <ipython-input-30-09eb1a8e88ef>:5(q1_time)
   117407    2.382    0.000    2.382    0.000 {orjson.loads}





In [34]:
!pip install memory-profiler
!python /content/q1_time_mp.py

Filename: /content/q1_time_mp.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7    117.0 MiB    117.0 MiB           1   @profile
     8                                         def q1_time(file_path: str) -> List[Tuple[datetime.date, str]]:
     9    117.0 MiB      0.0 MiB           1       data = []
    10    161.9 MiB      0.0 MiB           2       with open(file_path, 'rb') as f:
    11    161.9 MiB      1.5 MiB      117408           for line in f:
    12    161.9 MiB     34.0 MiB      117407               tweet = orjson.loads(line)
    13    161.9 MiB      9.3 MiB      234814               data.append({
    14    161.9 MiB      0.0 MiB      117407                   'date': tweet['date'],
    15    161.9 MiB      0.0 MiB      117407                   'username': tweet['user']['username']
    16                                                     })
    17                                         
    18    164.3 MiB      2.4 MiB           1       df = pd.DataFra

##### **Observaciones**

Parsear los datos con otra libreria, utilizar solo las columnas necesarias para armar el dataframe y utilizar las funciones nativas de pandas para calcular las agregaciones ayudo a reducir el tiempo de ejecucion a solo 3 seg,  incluso redujo el uso de memoria.


2. Los top 10 emojis más usados con su respectivo conteo. Debe incluir las siguientes funciones:
```python
def q2_time(file_path: str) -> List[Tuple[str, int]]:
```
```python
def q2_memory(file_path: str) -> List[Tuple[str, int]]:
```
```python
Returns:
[("✈️", 6856), ("❤️", 5876), ...]
```

In [38]:
import pandas as pd
import re
import orjson
from typing import List, Tuple

def q2_time(file_path: str) -> List[Tuple[str, int]]:
    data = []
    with open(file_path, 'rb') as f:
        for line in f:
            tweet = orjson.loads(line)
            data.append({
                'content': tweet['content'],
            })

    df = pd.DataFrame(data)

    contents = df['content']

    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        "\U0001F1E0-\U0001F1FF"  # Flags
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )

    def extract_emojis(text):
        return emoji_pattern.findall(text)

    df['emojis'] = contents.apply(extract_emojis)

    emojis_series = df['emojis'].explode()

    emojis_series = emojis_series.dropna()

    emoji_counts = emojis_series.value_counts()

    top_emojis = emoji_counts.head(10)

    result = list(top_emojis.items())

    return result

q2_time(file_path)

[('🙏', 1916),
 ('❤️', 952),
 ('😂', 627),
 ('🌾', 529),
 ('💚', 493),
 ('👍', 459),
 ('👉', 450),
 ('✊', 437),
 ('🇮🇳', 399),
 ('👇', 387)]

In [39]:
import cProfile
import pstats
import io

pr = cProfile.Profile()

pr.enable()
result = q2_time(file_path)
pr.disable()

s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('cumulative' )
ps.print_stats(5)
print(s.getvalue())


         706253 function calls (706219 primitive calls) in 4.733 seconds

   Ordered by: cumulative time
   List reduced from 341 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    4.733    2.367 /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3512(run_code)
        2    0.000    0.000    4.733    2.367 {built-in method builtins.exec}
        1    0.050    0.050    4.733    4.733 <ipython-input-39-6074579cebf1>:1(<cell line: 8>)
        1    0.882    0.882    4.683    4.683 <ipython-input-38-5c5a12b0916e>:6(q2_time)
   117407    1.978    0.000    1.978    0.000 {orjson.loads}





In [42]:
!pip install memory-profiler
!python /content/q2_time_mp.py

Filename: /content/q2_time_mp.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7    117.0 MiB    117.0 MiB           1   @profile
     8                                         def q2_time(file_path: str) -> List[Tuple[str, int]]:
     9    117.0 MiB      0.0 MiB           1       data = []
    10    184.1 MiB      0.0 MiB           2       with open(file_path, 'rb') as f:
    11    184.1 MiB      6.2 MiB      117408           for line in f:
    12    184.1 MiB     44.9 MiB      117407               tweet = orjson.loads(line)
    13    184.1 MiB     16.0 MiB      234814               data.append({
    14    184.1 MiB      0.0 MiB      117407                   'content': tweet['content'],
    15                                                     })
    16                                         
    17    186.1 MiB      2.1 MiB           1       df = pd.DataFrame(data)
    18                                         
    19    186.1 MiB      0.0 MiB           1    

In [43]:
import json
import re
from collections import defaultdict
from typing import List, Tuple

def q2_memory(file_path: str) -> List[Tuple[str, int]]:
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"  # Emoticons
        "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
        "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
        "\U0001F1E0-\U0001F1FF"  # Flags
        "\U00002702-\U000027B0"  # Dingbats
        "\U000024C2-\U0001F251"
        "]+",
        flags=re.UNICODE
    )

    emoji_counts = defaultdict(int)

    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                tweet = json.loads(line)
                content = tweet.get('content', '')
                emojis = emoji_pattern.findall(content)
                for emoji in emojis:
                    emoji_counts[emoji] += 1
            except json.JSONDecodeError:
                continue

    top_emojis = sorted(emoji_counts.items(), key=lambda x: x[1], reverse=True)[:10]

    return top_emojis

q2_memory(file_path)

[('🙏', 1916),
 ('❤️', 952),
 ('😂', 627),
 ('🌾', 529),
 ('💚', 493),
 ('👍', 459),
 ('👉', 450),
 ('✊', 437),
 ('🇮🇳', 399),
 ('👇', 387)]

In [44]:
import cProfile
import pstats
import io

pr = cProfile.Profile()

pr.enable()
result = q2_memory(file_path)
pr.disable()

s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('cumulative' )
ps.print_stats(5)
print(s.getvalue())


         1511575 function calls in 8.862 seconds

   Ordered by: cumulative time
   List reduced from 50 to 5 due to restriction <5>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    8.862    4.431 /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3512(run_code)
        2    0.000    0.000    8.862    4.431 {built-in method builtins.exec}
        1    0.000    0.000    8.862    8.862 <ipython-input-44-c1c9d4bd62b3>:1(<cell line: 8>)
        1    1.842    1.842    8.861    8.861 <ipython-input-43-badbe05d4749>:6(q2_memory)
   117407    0.229    0.000    5.878    0.000 /usr/lib/python3.10/json/__init__.py:299(loads)





In [46]:
!pip install memory-profiler
!python /content/q2_memory_mp.py

Filename: /content/q2_memory_mp.py

Line #    Mem usage    Increment  Occurrences   Line Contents
     7     38.2 MiB     38.2 MiB           1   @profile
     8                                         def q2_memory(file_path: str) -> List[Tuple[str, int]]:
     9     38.2 MiB      0.0 MiB           2       emoji_pattern = re.compile(
    10     38.2 MiB      0.0 MiB           1           "["
    11                                                 "\U0001F600-\U0001F64F"  # Emoticons
    12                                                 "\U0001F300-\U0001F5FF"  # Symbols & Pictographs
    13                                                 "\U0001F680-\U0001F6FF"  # Transport & Map Symbols
    14                                                 "\U0001F1E0-\U0001F1FF"  # Flags
    15                                                 "\U00002702-\U000027B0"  # Dingbats
    16                                                 "\U000024C2-\U0001F251"
    17                                       

#### **Observaciones**

Se utilizó la misma libreria, ya que la lectura del archivo es mas eficiente.
Probablemente se podia mejorar la funcion utilizando una libreria enfocada a detectar emojis en lugar de utilizar los unicode.